# Hyperparameter selection with Ray Tune

In [1]:
import os
from datetime import datetime
import time
import random
import warnings
import joblib
import warnings
import gc
# My modules
from config import Config
from logger import init_logger
from common_utils import set_seeds, read_csvs, stratify_split, setup, get_data_dfs, get_loaders
from model import Model
from train_loop_functions import train_epoch, valid_epoch

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score

import torch
from torch import nn
from torch.cuda.amp import GradScaler

# hyperparameter tuning
from functools import partial
import ray
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

In [2]:
%load_ext autoreload
%autoreload 2

# Setup

In [3]:
set_seeds(Config.seed)
LOGGER = init_logger() # uses Python's logging framework

# Tuning

In [4]:
# all params are passed in by Tune
def train_main(config, checkpoint_dir=None, data_dir=None):
    assert config is not None
    # -------- DATASETS AND LOADERS --------
    data_df, test_df = read_csvs(data_dir, Config.debug)
    train_folds = stratify_split(data_df, Config.fold_num, Config.seed, Config.target_col)
    
    # select only one of the folds (fold 0)
    train_df, valid_df = get_data_dfs(train_folds, 0)
    train_dataloader, valid_dataloader = get_loaders(train_df, valid_df,
                                                     config["batch_size"], 
                                                     data_dir+'/train_images')
    
    # -------- MODEL --------
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model, optimizer, scheduler, criterion = setup(Config.model_arch, config["lr"], 
                                                   config["is_amsgrad"], data_df.label.nunique(), device)
    
    # The `checkpoint_dir` parameter gets passed by Ray Tune when a checkpoint should be restored.
    if checkpoint_dir:
        checkpoint = os.path.join(checkpoint_dir, "checkpoint")
        model_state, optimizer_state = torch.load(checkpoint)
        model.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)
        
    # EPOCHS TRAIN
    for e in range(10):
        # TRAIN
        avg_training_loss = train_epoch(train_dataloader, model, 
                                      criterion, optimizer, 
                                      scheduler, GradScaler(), 
                                      config["accum_iter"], LOGGER,
                                      device)
        #print(torch.cuda.memory_summary(device))
        # VALIDATE
        avg_validation_loss, preds = valid_epoch(valid_dataloader, model, 
                                               criterion, config["accum_iter"],
                                               LOGGER, device)
  
        validation_labels = valid_df[Config.target_col].values
        accuracy = accuracy_score(y_true=validation_labels, y_pred=preds)
        
        # SAVE CHECKPOINT.
        # It is automatically registered with Ray Tune and will potentially
        # be passed as the `checkpoint_dir` parameter in future iterations.
        with tune.checkpoint_dir(step=e) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((model.state_dict(), optimizer.state_dict()), path)

        tune.report(loss=avg_validation_loss, accuracy=accuracy)
        gc.collect()

    print("Finished Training")

In [5]:
def main(num_samples=20, max_num_epochs=10, gpus_per_trial=1):
    data_dir = os.path.abspath('./data')
    
    hyperconfig = {
        "is_amsgrad": tune.choice([False, True]),
        "accum_iter": tune.choice([1,2,4,8,16]),
        "lr": tune.loguniform(1e-4, 1e-1),
        "batch_size": tune.choice([4, 8, 16])
    }
    
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    reporter = CLIReporter(metric_columns=["loss", "accuracy", "training_iteration"])
    result = tune.run(
        partial(train_main, data_dir=data_dir),
        resources_per_trial={"cpu": os.cpu_count(), "gpu": gpus_per_trial},
        config=hyperconfig,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter)

    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))
"""
    best_trained_model = Model(Config.model_arch, Config.num_labels, pretrained=True).to(device)
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if gpus_per_trial > 1:
            best_trained_model = nn.DataParallel(best_trained_model)
    best_trained_model.to(device)

    checkpoint_path = os.path.join(best_trial.checkpoint.value, "checkpoint")

    model_state, optimizer_state = torch.load(checkpoint_path)
    best_trained_model.load_state_dict(model_state)

    test_acc = test_accuracy(best_trained_model, device)
    print("Best trial test set accuracy: {}".format(test_acc))
"""

In [6]:
main()

2020-12-25 06:27:21,505	INFO services.py:1092 -- View the Ray dashboard at http://127.0.0.1:8265
2020-12-25 06:27:22,129	WARNING experiment.py:274 -- No name detected on trainable. Using DEFAULT.
2020-12-25 06:27:22,129	INFO registry.py:65 -- Detected unknown callable for trainable. Converting to class.
2020-12-25 06:27:22,161	WARNING worker.py:1091 -- The reporter on node Nevsky failed with the following error:
Traceback (most recent call last):
  File "/opt/favordata/anaconda3/envs/kaggle/lib/python3.7/site-packages/ray/reporter.py", line 309, in <module>
    reporter.run()
  File "/opt/favordata/anaconda3/envs/kaggle/lib/python3.7/site-packages/ray/reporter.py", line 245, in run
    port = server.add_insecure_port(f"[::]:{self.port}")
  File "/opt/favordata/anaconda3/envs/kaggle/lib/python3.7/site-packages/grpc/_server.py", line 962, in add_insecure_port
    address, _add_insecure_port(self._state, _common.encode(address)))
  File "/opt/favordata/anaconda3/envs/kaggle/lib/python3.7/

== Status ==
Memory usage on this node: 4.9/31.3 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 8/8 CPUs, 1/1 GPUs, 0.0/17.63 GiB heap, 0.0/6.05 GiB objects (0/1.0 accelerator_type:RTX)
Result logdir: /opt/favordata/ray_results/DEFAULT_2020-12-25_06-27-22
Number of trials: 1/20 (1 RUNNING)
+---------------------+----------+-------+--------------+--------------+--------------+-----------+
| Trial name          | status   | loc   |   accum_iter |   batch_size | is_amsgrad   |        lr |
|---------------------+----------+-------+--------------+--------------+--------------+-----------|
| DEFAULT_3efa2_00000 | RUNNING  |       |            1 |           16 | False        | 0.0122859 |
+---------------------+----------+-------+--------------+--------------+--------------+-----------+


(pid=20305) Epoch     0: adjusting learning rate of group 0 to 1.2286e-02.
Result for DEFAULT_3efa2_00000:
  a

Result for DEFAULT_3efa2_00000:
  accuracy: 0.28
  date: 2020-12-25_06-27-59
  done: false
  experiment_id: 1ad1c362cba44654bf6f108b767fbc2f
  experiment_tag: 0_accum_iter=1,batch_size=16,is_amsgrad=False,lr=0.012286
  hostname: Nevsky
  iterations_since_restore: 5
  loss: 7.100030183792114
  node_ip: 10.0.0.200
  pid: 20305
  should_checkpoint: true
  time_since_restore: 36.412052392959595
  time_this_iter_s: 6.529623746871948
  time_total_s: 36.412052392959595
  timestamp: 1608877679
  timesteps_since_restore: 0
  training_iteration: 5
  trial_id: 3efa2_00000
  
== Status ==
Memory usage on this node: 7.3/31.3 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: -9.361285209655762 | Iter 2.000: -4.071831822395325 | Iter 1.000: -2.007357656955719
Resources requested: 8/8 CPUs, 1/1 GPUs, 0.0/17.63 GiB heap, 0.0/6.05 GiB objects (0/1.0 accelerator_type:RTX)
Result logdir: /opt/favordata/ray_results/DEFAULT_2020-12-25_06-27-22
Number of trials: 2/20 (1 PENDING,

Result for DEFAULT_3efa2_00000:
  accuracy: 0.34
  date: 2020-12-25_06-28-32
  done: true
  experiment_id: 1ad1c362cba44654bf6f108b767fbc2f
  experiment_tag: 0_accum_iter=1,batch_size=16,is_amsgrad=False,lr=0.012286
  hostname: Nevsky
  iterations_since_restore: 10
  loss: 4.92010498046875
  node_ip: 10.0.0.200
  pid: 20305
  should_checkpoint: true
  time_since_restore: 69.57885503768921
  time_this_iter_s: 6.52949595451355
  time_total_s: 69.57885503768921
  timestamp: 1608877712
  timesteps_since_restore: 0
  training_iteration: 10
  trial_id: 3efa2_00000
  
== Status ==
Memory usage on this node: 7.3/31.3 GiB
Using AsyncHyperBand: num_stopped=1
Bracket: Iter 8.000: -6.290502548217773 | Iter 4.000: -9.361285209655762 | Iter 2.000: -4.071831822395325 | Iter 1.000: -2.007357656955719
Resources requested: 8/8 CPUs, 1/1 GPUs, 0.0/17.63 GiB heap, 0.0/6.05 GiB objects (0/1.0 accelerator_type:RTX)
Result logdir: /opt/favordata/ray_results/DEFAULT_2020-12-25_06-27-22
Number of trials: 2/20 

Result for DEFAULT_3efa2_00001:
  accuracy: 0.68
  date: 2020-12-25_06-29-14
  done: false
  experiment_id: 24b542e496264eed89ae5ceed32b35f3
  experiment_tag: 1_accum_iter=4,batch_size=4,is_amsgrad=False,lr=0.00047924
  hostname: Nevsky
  iterations_since_restore: 4
  loss: 1.3956382870674133
  node_ip: 10.0.0.200
  pid: 20308
  should_checkpoint: true
  time_since_restore: 39.4513099193573
  time_this_iter_s: 8.895934820175171
  time_total_s: 39.4513099193573
  timestamp: 1608877754
  timesteps_since_restore: 0
  training_iteration: 4
  trial_id: 3efa2_00001
  
== Status ==
Memory usage on this node: 7.1/31.3 GiB
Using AsyncHyperBand: num_stopped=1
Bracket: Iter 8.000: -6.290502548217773 | Iter 4.000: -5.3784617483615875 | Iter 2.000: -2.676600068807602 | Iter 1.000: -1.803763061761856
Resources requested: 8/8 CPUs, 1/1 GPUs, 0.0/17.63 GiB heap, 0.0/6.05 GiB objects (0/1.0 accelerator_type:RTX)
Result logdir: /opt/favordata/ray_results/DEFAULT_2020-12-25_06-27-22
Number of trials: 3/2

...

Result for DEFAULT_3efa2_00018:
  accuracy: 0.72
  date: 2020-12-25_06-40-43
  done: false
  experiment_id: b23a475ffec94e9b986f11a4ca253a5e
  experiment_tag: 18_accum_iter=4,batch_size=8,is_amsgrad=False,lr=0.0019013
  hostname: Nevsky
  iterations_since_restore: 8
  loss: 0.8204785883426666
  node_ip: 10.0.0.200
  pid: 30036
  should_checkpoint: true
  time_since_restore: 58.67919158935547
  time_this_iter_s: 6.876724481582642
  time_total_s: 58.67919158935547
  timestamp: 1608878443
  timesteps_since_restore: 0
  training_iteration: 8
  trial_id: 3efa2_00018
  
== Status ==
Memory usage on this node: 7.0/31.3 GiB
Using AsyncHyperBand: num_stopped=18
Bracket: Iter 8.000: -0.8767209947109222 | Iter 4.000: -0.98636394739151 | Iter 2.000: -1.3896448612213135 | Iter 1.000: -1.4405361711978912
Resources requested: 8/8 CPUs, 1/1 GPUs, 0.0/17.63 GiB heap, 0.0/6.05 GiB objects (0/1.0 accelerator_type:RTX)
Result logdir: /opt/favordata/ray_results/DEFAULT_2020-12-25_06-27-22
Number of trials:

Result for DEFAULT_3efa2_00018:
  accuracy: 0.72
  date: 2020-12-25_06-40-56
  done: true
  experiment_id: b23a475ffec94e9b986f11a4ca253a5e
  experiment_tag: 18_accum_iter=4,batch_size=8,is_amsgrad=False,lr=0.0019013
  hostname: Nevsky
  iterations_since_restore: 10
  loss: 0.8445485830307007
  node_ip: 10.0.0.200
  pid: 30036
  should_checkpoint: true
  time_since_restore: 72.36692214012146
  time_this_iter_s: 6.832549810409546
  time_total_s: 72.36692214012146
  timestamp: 1608878456
  timesteps_since_restore: 0
  training_iteration: 10
  trial_id: 3efa2_00018
  
== Status ==
Memory usage on this node: 7.0/31.3 GiB
Using AsyncHyperBand: num_stopped=19
Bracket: Iter 8.000: -0.8767209947109222 | Iter 4.000: -0.98636394739151 | Iter 2.000: -1.3896448612213135 | Iter 1.000: -1.4405361711978912
Resources requested: 8/8 CPUs, 1/1 GPUs, 0.0/17.63 GiB heap, 0.0/6.05 GiB objects (0/1.0 accelerator_type:RTX)
Result logdir: /opt/favordata/ray_results/DEFAULT_2020-12-25_06-27-22
Number of trials

2020-12-25 06:41:10,687	INFO tune.py:439 -- Total run time: 829.70 seconds (828.50 seconds for the tuning loop).


Result for DEFAULT_3efa2_00019:
  accuracy: 0.54
  date: 2020-12-25_06-41-10
  done: true
  experiment_id: 21950de8df5d49c683d5631d633905a0
  experiment_tag: 19_accum_iter=1,batch_size=4,is_amsgrad=False,lr=0.067995
  hostname: Nevsky
  iterations_since_restore: 1
  loss: 795.1906127929688
  node_ip: 10.0.0.200
  pid: 4989
  should_checkpoint: true
  time_since_restore: 11.869887590408325
  time_this_iter_s: 11.869887590408325
  time_total_s: 11.869887590408325
  timestamp: 1608878470
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 3efa2_00019
  
== Status ==
Memory usage on this node: 6.9/31.3 GiB
Using AsyncHyperBand: num_stopped=20
Bracket: Iter 8.000: -0.8767209947109222 | Iter 4.000: -0.98636394739151 | Iter 2.000: -1.3896448612213135 | Iter 1.000: -1.4763555526733398
Resources requested: 8/8 CPUs, 1/1 GPUs, 0.0/17.63 GiB heap, 0.0/6.05 GiB objects (0/1.0 accelerator_type:RTX)
Result logdir: /opt/favordata/ray_results/DEFAULT_2020-12-25_06-27-22
Number of trials: 

AttributeError: type object 'Config' has no attribute 'num_labels'

In [ ]:
#^ignore error